# Segmenting and Clustering Neighborhoods in Toronto

This is a peer graded assignment I have from the IBM Data Science Capstone. 

In this assignment, I explored, segmented, and clustered the neighborhoods in the city of Toronto based on the postalcode and borough information.

### Part 1: Scraping the web for neighborhood data in Toronto

A Wikipedia page ("List of postal codes of Canada: M") that has all the information for the neighborhoods in Toronto is used.

The data is then cleaned, read it into a pandas  dataframe so that it is in a structured format.

In [1]:
#Data processing libraries
import pandas as pd
import numpy as np
import random

from bs4 import BeautifulSoup #library to scrape the web
import requests # library to handle requests


In [2]:
#Extracting html content from wikipedia
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
soup = BeautifulSoup(data,'html5lib')
table=soup.find('table') #Tables only

**Sample HTML for the cell**

</td>
<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>

In [3]:
# We want to extract the 'Postal Code', 'Borough', 'Neighborhood' from the page
l_address = []

for row in table.find_all('tr'): # rows of the main table
    for cell in row.find_all('td'): # each row is made up of multiple cell
        if cell.span.text.split('(')[0]=='Not assigned': 
            pass #If the cell is not assigned Neighborhood, pass through to avoid error
        else:
            Postal_Code = (cell.b.text)
            Borough = (cell.span.text.split('(')[0])
            Neighborhood = ((((((cell.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' '))
            l_address.append([Postal_Code,Borough,Neighborhood])


In [4]:
df_address=pd.DataFrame(l_address)
df_address.columns = ['Postal Code', 'Borough', 'Neighborhood']

In [5]:
df_address.head(11)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
df_address.shape

(103, 3)

### Part 2: Use the Geocoder package to obtain the lat and long for each neighborhood

In [18]:
import geocoder # import geocoder

latitude,longitude = [],[] # Lists to store the results

for PostalCode in df_address['Postal Code']:
    lat_lng_coords = None # initialize variable to None
    while(lat_lng_coords is None):     # loop until get the coordinates
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(PostalCode))
        lat_lng_coords = g.latlng
    #print (lat_lng_coords[0],' ',lat_lng_coords[1])
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

In [19]:
df_address['Latitude'] = latitude
df_address['Longitude'] = longitude
df_address.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188
5,M9A,Etobicoke,Islington Avenue,43.66263,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,M3B,North York,Don Mills North,43.74923,-79.36186
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


### Part 3: Segmenting the Clustering Neighborhoods in Downtown Toronto 
- Data Source: **four square API**
- Algorithm: **Kmeans**
- Visualization: **Folium**

#### Sourcing the data

In [27]:
#Prepare the address data
TorontoData = df_address[df_address['Borough'] == 'Downtown Toronto'].reset_index(drop=True)

In [48]:
import config #Four square API key
VERSION = '20180604'
LIMIT = 100
radius = 500

In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
toronto_venues = getNearbyVenues(names=TorontoData['Neighborhood'],
                                   latitudes=TorontoData['Latitude'],
                                   longitudes=TorontoData['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [52]:
print(toronto_venues.shape)
toronto_venues.head()

(1096, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


In [59]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Theater,Theme Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.015873,0.000000,0.000000,0.000000,0.015873,0.0,0.015873,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.015873,0.000000,0.000000,0.000000,0.000000,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.013158,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.013158,0.000000,0.000000,0.000000,0.0
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.016129,0.016129,0.0,...,0.016129,0.000000,0.0,0.0,0.000000,0.000000,0.016129,0.016129,0.016129,0.0
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Church and Wellesley,0.011905,0.011905,0.011905,0.011905,0.000000,0.0,0.000000,0.000000,0.0,...,0.011905,0.011905,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


#### Algorithm: Kmeans

In [63]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.06
2  Seafood Restaurant  0.05
3              Bakery  0.05
4         Cheese Shop  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.07
2                Café  0.05
3                 Bar  0.04
4                Park  0.04


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.11
1             Clothing Store  0.06
2                       Bank  0.03
3  Middle Eastern Restaurant  0.03
4                       Café  0.03


----Christie----
                venue  freq
0                Café  0.27
1       Grocery Store  0.18
2          Playground  0.09
3  Italian Restaurant  0.09
4         Candy Store  0.09


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.11
1  Japanese 

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Pharmacy,Breakfast Spot,Restaurant,Beer Bar,Farmers Market,Cheese Shop
1,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,French Restaurant,Park,Bar,Spa,Bakery,Speakeasy,Restaurant
2,Central Bay Street,Coffee Shop,Clothing Store,Bank,Sandwich Place,Café,Plaza,Bubble Tea Shop,Restaurant,Cosmetics Shop,Middle Eastern Restaurant
3,Christie,Café,Grocery Store,Playground,Candy Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Baby Store,Dessert Shop,Diner
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Hotel,Café,Men's Store,Mediterranean Restaurant,Dance Studio


In [69]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = TorontoData

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,1,Coffee Shop,Breakfast Spot,Restaurant,Health Food Store,Spa,Bakery,Distribution Center,Electronics Store,Event Space,Food Truck
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop,Italian Restaurant,Hotel,Middle Eastern Restaurant,Diner,Bookstore
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,1,Coffee Shop,Cocktail Bar,Café,Italian Restaurant,Clothing Store,Hotel,Gastropub,Cosmetics Shop,Gym,American Restaurant
3,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Pharmacy,Breakfast Spot,Restaurant,Beer Bar,Farmers Market,Cheese Shop
4,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493,1,Coffee Shop,Clothing Store,Bank,Sandwich Place,Café,Plaza,Bubble Tea Shop,Restaurant,Cosmetics Shop,Middle Eastern Restaurant


#### Visualization: Folium

In [79]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library


# create map
map_clusters = folium.Map(location=[43.65512, -79.36264], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters